<a href="https://colab.research.google.com/github/Trek3/stacksample-amd/blob/main/stacksample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets files stackoverflow/stacksample
!kaggle datasets download stackoverflow/stacksample -p .kaggle/ -f Questions.csv
!cd .kaggle
!unzip Questions.csv.zip

name           size  creationDate         
-------------  ----  -------------------  
Answers.csv     1GB  2019-10-08 00:37:26  
Questions.csv   2GB  2019-10-08 00:37:26  
Tags.csv       62MB  2019-10-08 00:37:26  
Questions.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open Questions.csv.zip, Questions.csv.zip.zip or Questions.csv.zip.ZIP.


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with open('.kaggle/Questions.csv', 'rb') as f:
  df = pd.read_csv(f, sep=',', usecols=['Id','Body'], encoding='ISO-8859-1', index_col=0).head(100)

df

/home/trekkete/.local/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Body
Id,
80,<p>I've written a database generation script i...
90,<p>Are there any really good tutorials explain...
120,<p>Has anyone got experience creating <strong>...
180,<p>This is something I've pseudo-solved many t...
260,<p>I have a little game written in C#. It uses...
...,...
9570,<p>I'm integrating .NET support into our C++ a...
9650,<p><br />\nI've been wanting to teach myself L...
9750,<p>I was reading Joel's book where he was sugg...


In [2]:
import random as rd

def build_hashrow_functions(n, k):
  ixs = list(range(n))
  rd.shuffle(ixs)
  return [lambda x, i=i : (x + i) % k for i in ixs]

In [18]:
def shingle(k, body):
  return [body[i:i+k] for i in range(len(body) // k)]

import functools

def get_all_shingles(k):
  return set(functools.reduce(lambda a, b : a+b, [shingle(k, df.iloc[i]['Body']) for i in range(len(df))]))

all_shingles = sorted(get_all_shingles(4))

shingles_per_column = [shingle(4, df.iloc[i]['Body']) for i in range(len(df))]

charmat = [[1 if shin in col_i else 0 for col_i in shingles_per_column] for shin in all_shingles]
fs = build_hashrow_functions(100, len(charmat))


In [70]:

#fs = [lambda x : (3*x + 1) % len(charmat), lambda x : (x + 1) % len(charmat)]
#print([[f(i) for i in range(11)] for f in fs])

sig = [[len(charmat) + 1 for _ in range(len(df))] for _ in range(len(fs))]

for i in range(len(charmat)):
  indexes = [f(i) for f in fs]
  for j in range(len(charmat[i])):
    if charmat[i][j] == 1:
      for l in range(len(sig)):
        sig[l][j] = min(sig[l][j], indexes[l])

def sim(s1, s2):
  return sum([1 if x1 == x2 else 0 for (x1, x2) in zip(s1, s2)])/(len(s1))


In [68]:
sigdf = pd.DataFrame(sig)
sigdf

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,75,227,47,3,75,16,4,46,2,6,...,76,3,34,11,427,99,60,329,343,459
1,23,175,23,267,23,176,148,2,119,138,...,24,80,381,164,375,47,8,277,291,407
2,38,190,10,282,38,1,163,9,0,7,...,39,95,396,5,390,62,23,292,306,422
3,4,156,4,248,4,157,129,4,100,119,...,5,61,362,145,356,28,24,258,272,388
4,60,212,32,304,60,1,185,31,22,29,...,61,117,19,27,412,84,45,314,328,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,46,198,18,290,46,9,171,17,8,15,...,47,103,5,13,398,70,31,300,314,430
96,13,165,13,257,13,166,138,13,109,128,...,14,70,371,154,365,37,33,267,281,397
97,21,173,21,265,21,174,146,0,117,136,...,22,78,379,162,373,45,6,275,289,405
98,95,15,67,23,95,36,24,9,1,7,...,6,23,54,31,447,119,80,349,9,479


In [71]:
pd.DataFrame([[sim(sigdf[row], sigdf[col]) for col in sigdf] for row in sigdf])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.00,0.0,0.24,0.0,0.92,0.0,0.0,0.17,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,1.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.24,0.0,1.00,0.0,0.24,0.0,0.0,0.17,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.00,1.0,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.92,0.0,0.24,0.0,1.00,0.0,0.0,0.17,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
96,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
97,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
98,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.18,0.08,0.0,...,0.03,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [58]:
print('FIRST: \n\n', df.iloc[0]['Body'], '\n\nSECOND: \n\n', df.iloc[4]['Body'])

FIRST: 

 <p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thumbnailID integer
    ,fileFormatID integer
    ,categoryID integer
    ,isFavorite boolean
    ,dateAdded date
    ,globalAccessCount integer
    ,lastAccessTime date
    ,downloadComplete boolean
    ,isNew boolean
    ,isSpotlight boolean
    ,duration varchar(30)
);
Create Table tCategory (
    categoryID integer Primary Key
    ,categoryName varchar(50)
    ,parent_categoryID integer
);
...
</code></pre>

<p>I execute this in Adobe AIR using the following methods:</p>

<pre><code>public static function RunSqlFromFile(fileName:String):void {

In [59]:
s91 = shingle(4, df.iloc[0]['Body'])
s3 = shingle(4, df.iloc[4]['Body'])

[s for s in s91 if s in s3]

['<p>I',
 'e wr',
 ' wri',
 'writ',
 'ritt',
 'itte',
 'tten',
 'ten ',
 ' a d',
 'a da',
 ' dat',
 'data',
 'atab',
 'taba',
 'abas',
 'base',
 'ase ',
 'tion',
 'ion ',
 ' scr',
 'scri',
 'crip',
 'ript',
 ' in ',
 ' <a ',
 '<a h',
 'a hr',
 ' hre',
 'href',
 'ref=',
 'ef="',
 'f="h',
 '="ht',
 '"htt',
 'http',
 'ttp:',
 'tp:/',
 'p://',
 '://e',
 '//en',
 '/en.',
 'en.w',
 'n.wi',
 '.wik',
 'wiki',
 'ikip',
 'kipe',
 'iped',
 'pedi',
 'edia',
 'dia.',
 'ia.o',
 'a.or',
 '.org',
 'org/',
 'rg/w',
 'g/wi',
 '/wik',
 'wiki',
 'iki/',
 '</a>',
 ' and',
 'and ',
 ' wan',
 'want',
 'nt t',
 ' to ',
 ' in ',
 ' <a ',
 '<a h',
 'a hr',
 ' hre',
 'href',
 'ref=',
 'ef="',
 'f="h',
 '="ht',
 '"htt',
 'http',
 'ttp:',
 'tp:/',
 'p://',
 '://e',
 '//en',
 '/en.',
 'en.w',
 'n.wi',
 '.wik',
 'wiki',
 'ikip',
 'kipe',
 'iped',
 'pedi',
 'edia',
 'dia.',
 'ia.o',
 'a.or',
 '.org',
 'org/',
 'rg/w',
 'g/wi',
 '/wik',
 'wiki',
 'iki/',
 'me">',
 '</a>',
 'tion',
 '</p>',
 '/p>\n',
 'p>\n\n',
 '>\n\n

In [50]:
a = 1
b = 1
a ^ b

0